This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

# Step 1: Setting Up the Python Application
Initialize a Python Project: 
- [*] Create a new Python project
    git
- [*] Setting up a virtual environment and 
    Conda
- [*] Installing necessary packages like LangChain, 
    requirements.yml
- [*] A suitable LLM library (e.g., OpenAI's GPT)
    OpenAi
- [*] A vector database package compatible with Python (e.g., ChromaDB or LanceDB). 
    ChromaDB

In [6]:
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
import chromadb
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables.passthrough import RunnablePassthrough

from dotenv import load_dotenv
import os
load_dotenv()  # take environment variables from .env.

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
MODEL_NAME = "gpt-3.5-turbo"
TEMPERATURE = 0.1

# Step 2: Generating Real Estate Listings
- [*] Generate real estate listings using a Large Language Model.
- [*] Generate at least 10 listings This can involve creating prompts for the LLM to produce descriptions of various properties

## Create Prompt

In [7]:
request_listing_examples = """
Generate {number} real estate listings seperated by (###) for {location}.
Your answer must be in the format of the following example:

###
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft
Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.
Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
###

"""

prompt = PromptTemplate(input_variables=["number", "location"], template=request_listing_examples)
print(prompt.format_prompt(number = 15 , location = "COVENTRY, ENGLAND"))

text='\nGenerate 15 real estate listings seperated by (###) for COVENTRY, ENGLAND.\nYour answer must be in the format of the following example:\n\n###\nNeighborhood: Green Oaks\nPrice: $800,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 2,000 sqft\nDescription: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.\nNeighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or gr

## Call LLM

In [8]:
llm = ChatOpenAI(
    model_name=MODEL_NAME,
    temperature=TEMPERATURE,
    max_tokens = 4096,
    openai_api_key=OPENAI_API_KEY
)

chain = prompt | llm | StrOutputParser()

# Request Listing
response = chain.invoke(
    {
        "number": 15, 
        "location":"COVENTRY, ENGLAND"
    }
)


## Save Listings to file

In [9]:

# Write to file
f = open("listings.txt", "a")
f.writelines(response)
f.close()

# Step 3: Storing Listings in a Vector Database
- [*] Vector Database Setup: Initialize and configure ChromaDB or a similar vector database to store real estate listings.
- [*] Generating and Storing Embeddings: Convert the LLM-generated listings into suitable embeddings that capture the semantic content of each listing, and store these embeddings in the vector database.

In [10]:
loader = TextLoader(file_path='listings.txt')
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
db = Chroma.from_documents(split_docs, embeddings)
retriever = db.as_retriever()

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## Test Symantic Query

In [11]:
query = "three-bedroom"
docs = db.similarity_search(query)
print(len(docs))
print(docs[0].page_content)

4
###
Neighborhood: Cheylesmore
Price: $550,000
Bedrooms: 3
Bathrooms: 2
House Size: 1,600 sqft
Description: This charming 3-bedroom bungalow in Cheylesmore offers single-level living with modern updates. The open-concept living and dining area features hardwood floors and a cozy fireplace, while the updated kitchen boasts granite countertops and stainless steel appliances. The master suite includes a walk-in closet and en-suite bathroom, while the landscaped backyard is perfect for outdoor entertaining. With easy access to shops, schools, and parks, this bungalow is a must-see.


# Step 4: Building the User Preference Interface
- [*] Collect buyer preferences, such as the number of bedrooms, bathrooms, location, and other specific requirements from a set of questions or telling the buyer to enter their preferences in natural language. 
- [*] You can hard-code the buyer preferences in questions and answers, or collect them interactively however you'd like, example:


In [12]:
questions = [   
    "How big do you want your house to be?" 
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",   
]
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

# Step 5: Searching Based on Preferences
- [*] Semantic Search Implementation: Use the structured buyer preferences to perform a semantic search on the vector database, retrieving listings that most closely match the user's requirements.
- [*] Listing Retrieval Logic: Fine-tune the retrieval algorithm to ensure that the most relevant listings are selected based on the semantic closeness to the buyer’s preferences.

In [14]:
docs = []
for answer in answers:
    docs.extend(db.similarity_search(answer))

# Step 6: Personalizing Listing Descriptions
- [ ] LLM Augmentation: For each retrieved listing, use the LLM to augment the description, tailoring it to resonate with the buyer’s specific preferences. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for.
- [ ] Maintaining Factual Integrity: Ensure that the augmentation process enhances the appeal of the listing without altering factual information.


In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
                You are a professional real estate agent.
                Answer all questions to the best of your ability using accurate data.
                For each  listing in the Context, augment the description, tailoring it to resonate with the buyer’s specific preferences. 
                This involves subtly emphasizing aspects of the property that align with what the buyer is looking for
                Context :{context}
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{question}")
    ]
)

prompt.format(context="Bannanas", question="Apples" , messages = [("user","BOOOO")])

'System: \n                You are a professional real estate agent.\n                Answer all questions to the best of your ability using accurate data.\n                For each  listing in the Context, augment the description, tailoring it to resonate with the buyer’s specific preferences. \n                This involves subtly emphasizing aspects of the property that align with what the buyer is looking for\n                Context :Bannanas\n            \nHuman: BOOOO\nHuman: Apples'

In [30]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chat_history = ChatMessageHistory()

for q,a in zip(questions,answers):
    chat_history.add_ai_message(q)
    chat_history.add_user_message(a)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough(), "messages": lambda x: chat_history.messages}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
response = rag_chain.invoke("Get tailord list")

In [39]:
print(response)

For your consideration:

1. **Property in Earlsdon**
   - **Neighborhood:** Earlsdon
   - **Price:** $700,000
   - **Bedrooms:** 4
   - **Bathrooms:** 3
   - **House Size:** 2,500 sqft
   - **Description:** Step into this charming 4-bedroom Victorian home in Earlsdon, where historic charm meets modern updates. With a spacious kitchen perfect for your culinary adventures, a cozy living room for relaxation, and a beautifully landscaped garden for your gardening aspirations, this property offers the comfort and space you desire. Enjoy the convenience of nearby shops and cafes, along with easy access to public transportation for your daily commute.

2. **Property in Cheylesmore**
   - **Neighborhood:** Cheylesmore
   - **Price:** $550,000
   - **Bedrooms:** 3
   - **Bathrooms:** 2
   - **House Size:** 1,600 sqft
   - **Description:** Nestled in the heart of Cheylesmore, this charming 3-bedroom bungalow offers single-level living with modern updates. The spacious kitchen with granite counte

# Step 7: Deliverables and Testing
- [ ] Test your "HomeMatch" application and make sure it meets all of the requirements in the rubric(opens in a new tab). Your project code will be run when it's assessed. Enter different "buyer preferences" and ensure it works.
- [ ] Jupyter Notebook/Python Program: Compile the application code in a Jupyter notebook or a standalone Python program. Ensure the code is well-commented and logically structured.
- [ ] Example Outputs: Include example outputs showcasing how user preferences are processed and how the application generates personalized listing descriptions. You can include these in comments in your application or in a Jupyter notebook that's saved with outputs.

You can see from the results above, from the users questions we have returned a list of suitable properties. Then we tailord the description to the users answers to the questions.
A specific example from above is:
- `With a spacious kitchen perfect for your culinary adventures, a cozy living room for relaxation, and a beautifully landscaped garden for your gardening aspirations`

Which matches the users preferences:
- `A comfortable three-bedroom house with a spacious kitchen and a cozy living room.`
- `A backyard for gardening`

# Step 8: Project Submission
- [ ] Generated Listings: Include a file that contains your synthetically generated real estate listings. Name this file "listings"
- [ ] Project Documentation: Include a readme file or an accompanying document explaining the functionality, how to run the code, and any prerequisites or dependencies.
- [ ] Code Submission: Submit the Jupyter Notebook or Python program on the "Project Submission Page" that follows the workspace page.